In [2]:
import sys
#!{sys.executable} -m pip install kafka-python
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install geojson
#!{sys.executable} -m pip install -I setuptools 
#!{sys.executable} -m pip install matplotlib

  Using cached setuptools-59.6.0-py3-none-any.whl (952 kB)


In [3]:
## https://medium.com/@technologydata25/connect-neo4j-to-jupyter-notebook-c178f716d6d5
from py2neo import Graph,Node,Relationship

In [12]:
url="bolt://44.203.177.87:7687"
username="neo4j"
password="animals-art-folder"

In [13]:
graph = Graph(url, auth=(username, password))

In [14]:
graph

Graph('bolt://44.203.177.87:7687')

In [16]:
createConstraintQuery="""
create constraint pkCity
for (c:City)
require c.cityName is unique
"""
graph.evaluate(createConstraintQuery)

In [32]:
populateDbQuery="""
call apoc.periodic.iterate(
"load csv with headers from 'https://raw.githubusercontent.com/rennyatwork/GeoPlot/master/DataSets/ticketToRide_edges.csv' as row
fieldterminator ';'
with row
return row"
, "with row
merge (orig:City{
cityName:row.orig
, lat:apoc.convert.toFloat(row.orig_lat) 
, lon:apoc.convert.toFloat(row.orig_lon) 
})

merge (dest:City{
cityName:row.dest
, lat:apoc.convert.toFloat(row.dest_lat) 
, lon:apoc.convert.toFloat(row.dest_lon) 
})

merge (orig)-[:is_connected{distance:toInteger(row.distance), color:row.color}]-(dest)
"
, {batchSize:100, parallel:false}
)
"""
graph.evaluate(populateDbQuery)

1

In [33]:
x=graph.evaluate("match(n) return count(n)")

In [34]:
x

35

In [35]:
x=graph.evaluate("match(n) return n")

In [37]:
type(x)

py2neo.data.Node